Simon Camilo Baez

Project 3 Data Analytics

Redacted

Profitable ad suggestions for devs - This notebook shows devs what kind of mobile apps can make them the most money (as of 2018). This notebook is designed for devs that want to make free apps that profit from advertisements, and targets an english speaking audience.

In [142]:
from csv import reader

## opens the Android (Google Play) dataset
opening = open('googleplaystore2018.csv', encoding='utf8')
read = reader(opening)
android = list(read)
header_android = android[0] ## this is the header
android = android[1:] ## This is the data without the header
del android[10472]
del android[9148]

## opens the IOS (App Store) dataset
opening = open('applestore2017.csv', encoding='utf8')
read = reader(opening)
apple = list(read)
header_apple = apple[0] ## this is the header
apple = apple[1:] ## This is the data without the header

## Below is a function to explore the datasets and see what data we are dealing with.

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
    if rows_and_columns:
        print('\n') # adds a new (empty) line after it finishes printing rows
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [143]:
print(header_android)
print("\n")
explore_data(android, 100, 104, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Natural recipes for your beauty', 'BEAUTY', '4.7', '1150', '9.8M', '100,000+', 'Free', '0', 'Everyone', 'Beauty', '15-May-18', '4', '4.1 and up']
['BestCam Selfie-selfie, beauty camera, photo editor', 'BEAUTY', '3.9', '1739', '21M', '500,000+', 'Free', '0', 'Everyone', 'Beauty', '12-Jul-18', '1.0.6', '4.0.3 and up']
['Mirror - Zoom & Exposure -', 'BEAUTY', '3.9', '32090', 'Varies with device', '1,000,000+', 'Free', '0', 'Everyone', 'Beauty', '24-Oct-16', 'Varies with device', 'Varies with device']
['Beauty Selfie Camera', 'BEAUTY', '4.2', '2225', '52M', '500,000+', 'Free', '0', 'Everyone', 'Beauty', '28-Feb-18', '1.6', '4.1 and up']


Number of rows: 10839
Number of columns: 13


In [144]:
print(header_apple)
print("\n")
explore_data(apple, 50, 53, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['310738695', 'Zillow Real Estate - Homes for Sale & for Rent', '132632576', 'USD', '0', '342969', '88478', '4.5', '4.5', '10.4.5', '4+', 'Lifestyle', '37', '5', '1', '1']
['305343404', 'Tumblr', '151573504', 'USD', '0', '334293', '919', '4', '4', '8.6', '17+', 'Social Networking', '37', '5', '16', '1']
['403858572', 'Fruit Ninja®', '163801088', 'USD', '0', '327025', '82', '4.5', '4', '2.5.1', '4+', 'Games', '38', '5', '13', '1']


Number of rows: 7197
Number of columns: 16


Using the code below, we can see that the dataset for google play has duplicates. We need to remove these duplicates before we can proceed.

In [145]:
duplicate_apps = []
unique_apps = []

## finds duplicates
for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


We need to keep the version of the duplicated data that's the most accurate, this should be the most recent duplicate since it will have the most up to date rating. 

In [146]:
reviews_max = {}

## finds duplicates, and removes the ones that have less reviews
for app in android: 
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews: 
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

## Does the same thing I guess?? I need to consult this with my professor

android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

explore_data(android_clean, 9147, 9150, True)

['Comunidad De Fe Minitries', 'LIFESTYLE', '5', '19', '11M', '500+', 'Free', '0', 'Everyone', 'Lifestyle', '1-Mar-18', '1.1', '4.1 and up']
['Garena Free Fire', 'GAME', '4.5', '5534114', '53M', '100,000,000+', 'Free', '0', 'Teen', 'Action', '3-Aug-18', '1.21.0', '4.0.3 and up']
['FE Other Disciplines Engineering Exam Prep', 'FAMILY', 'NaN', '0', '21M', '100+', 'Free', '0', 'Everyone', 'Education', '27-Jul-18', '5.33.3669', '5.0 and up']


Number of rows: 9658
Number of columns: 13


In [147]:
android_Eng = []
ios_Eng = []

for app in android_clean:
    n = 0
    for letter in app[0]:
        letter = ord(letter)
        if letter > 127:
            n += 1
    if n < 4:
            android_Eng.append(app)

for app in apple:
    n = 0
    for letter in app[1]:
        letter = ord(letter)
        if letter > 127:
            n += 1
    if n < 4:
            ios_Eng.append(app)
            
explore_data(ios_Eng, 5632, 5635, True)

['668576857', 'Mister Maker: Let’s Make It! – Design, Draw, Paint, Make and Play', '181036032', 'USD', '2.99', '9', '0', '4.5', '0', '6', '4+', 'Entertainment', '40', '5', '1', '1']
['1148046351', 'Cosmolander - Missions in the Solar System', '369460224', 'USD', '2.99', '9', '0', '2.5', '0', '2.0.1', '4+', 'Education', '37', '5', '6', '1']
['1179475725', 'Peppa Pig: Party Time', '251977728', 'USD', '0.99', '9', '0', '2', '0', '1.1.6', '4+', 'Education', '40', '5', '1', '1']


Number of rows: 6183
Number of columns: 16


In [148]:
android = []
apple = []

for app in android_Eng:
    if app[7] == "0":
        android.append(app)

for app in ios_Eng:
    if app[4] == "0":
        apple.append(app)

explore_data(apple, 180, 183, True)

['564540143', 'Hill Climb Racing', '108322816', 'USD', '0', '108183', '186', '4.5', '4.5', '1.33.0', '9+', 'Games', '37', '5', '1', '1']
['297430070', 'Target', '138754048', 'USD', '0', '108131', '13', '3', '2', '8.21.0', '4+', 'Shopping', '37', '0', '1', '1']
['667728512', 'Game of War - Fire Age', '220386304', 'USD', '0', '107916', '507', '4', '3', '3.22.510', '9+', 'Games', '37', '5', '29', '1']


Number of rows: 3222
Number of columns: 16


To avoid wasting time and money on a app that may not get enough installs, we first make an app that has minimal features and provides a concept, just to see if people like the idea. If people like it, we spend more time on the app and develop it further so it's more fleshed out. If the app is profitable after 6 months, we make a version for the App Store, so we can increase our profits since the app is going to be available to more people. Because of we are going to make a version of the app for the App Store, we also need to make sure that we appeal to people in the App Store.

To find what kind of app is going to be the most profitable, we need to see what genres get more attention in both stores. We are going to use the Category and Genre column on Google play (column 2 & 10), and the Prime Genre column on the app store (column 12).

In [149]:
def freq_table(dataset, index, filterer): ## Gets percentages that can be analyzed later
    data = {}
    new_data = {}
    total = 0
    if dataset == android: ## The filterer is multiplied by 100 to simplify printing the table, since there are a lot of values for android. In apple, there are not enough apps, so the table is not simplified.
        filterer *= 100
    elif dataset == apple:
        filterer *= 10
    ## Note for professor: There is probably a way to do this in a way that works with any data sample of any size, but it would take a lot more time and testing that I would have to do myself.
    ## If you see this I probably ran out of time to do that. It's fine anyways, I probably shouldn't even spend so much time trying to do these nitpicks. You could say this answers the questions on Data Analysis (Step 4 Part 3 on my summarized doc). I did answer them on the doc, not sure if I should answer them here
    for row in dataset:
        total += 1
        value = row[index]
        if value in data:
            data[value] += 1
        else:
            data[value] = 1
    for row in data: ## this function removes categories that are irrelevant, so the data is easier to go through. If there are less apps in a category than the filterer, then those apps won't be included
        if data[row] > filterer:
            new_data[row] = data[row]
        else:
            total -= data[row]
    
    data = new_data
    percentages = {}
    for row in data:
        percentage = (data[row] / total) * 100
        percentages[row] = percentage
    return percentages

def display_table(dataset, index, minpercent): ## Uses the definition from above to organize the percentages, and make them easier to read
    table = freq_table(dataset, index, minpercent) ## this is where the definition from before gets referenced
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', round(entry[0], 2), end="%\n")

print("Apple Prime Genre")
display_table(apple, 11, 6) #dataset(android, apple), column of genre(number), sets strenght of filter, the higher the less results(1-digit number)

Apple Prime Genre
Games : 65.14%
Entertainment : 8.83%
Photo & Video : 5.56%
Education : 4.1%
Social Networking : 3.68%
Shopping : 2.92%
Utilities : 2.82%
Sports : 2.4%
Music : 2.29%
Health & Fitness : 2.26%


In [150]:
for genre in freq_table(apple, 11, 0):
    total = 0
    len_genre = 0
    for app in apple:
        genre_app = app[11]
        if genre_app == genre:
            total += float(app[5])
            len_genre += 1
    average = total / len_genre
    print(genre, round(average, 1))

Social Networking 71548.3
Photo & Video 28441.5
Games 22788.7
Music 57326.5
Reference 74942.1
Health & Fitness 23298.0
Weather 52279.9
Utilities 18684.5
Travel 28243.8
Shopping 26919.7
News 21248.0
Navigation 86090.3
Lifestyle 16485.8
Entertainment 14029.8
Food & Drink 33333.9
Sports 23008.9
Book 39758.5
Finance 31467.9
Education 7004.0
Productivity 21028.4
Business 7491.1
Catalogs 4004.0
Medical 612.0


In [151]:
for genre in freq_table(android, 1, 0):
    total = 0
    len_category = 0
    for category in android:
        category_app = category[1]
        if category_app == genre:
            category[5] = category[5].replace("+", "")
            category[5] = category[5].replace(",", "")
            total += float(category[5])
            len_category += 1

    average = total / len_category
    print(genre, round(average, 1))

ART_AND_DESIGN 1986335.1
AUTO_AND_VEHICLES 647317.8
BEAUTY 513151.9
BOOKS_AND_REFERENCE 8767811.9
BUSINESS 1712290.1
COMICS 817657.3
COMMUNICATION 38456119.2
DATING 854028.8
EDUCATION 1833495.1
ENTERTAINMENT 11640705.9
EVENTS 253542.2
FINANCE 1387692.5
FOOD_AND_DRINK 1924897.7
HEALTH_AND_FITNESS 4188822.0
HOUSE_AND_HOME 1331540.6
LIBRARIES_AND_DEMO 638503.7
LIFESTYLE 1437816.3
GAME 15588015.6
FAMILY 3697848.2
MEDICAL 120550.6
SOCIAL 23253652.1
SHOPPING 7036877.3
PHOTOGRAPHY 17840110.4
SPORTS 3638640.1
TRAVEL_AND_LOCAL 13984077.7
TOOLS 10801391.3
PERSONALIZATION 5201482.6
PRODUCTIVITY 16787331.3
PARENTING 542603.6
WEATHER 5074486.2
VIDEO_PLAYERS 24727872.5
NEWS_AND_MAGAZINES 9549178.5
MAPS_AND_NAVIGATION 4056941.8


Conclusions

An application to take pictures could be very profitable. These applications tend to get a lot of installs, and while there is a lot of competition, we can still stand out. In order to stand out, the application could provide versatility in how their pictures are created. The app can also provide filters, and could provide effects that appear on screen. Today we have the technology of making our devices detect our facial expresions, which could be very useful in making effects that interact with the user, or imitate the user. The app could also work as a photo editor, so final touches could be applied in the picture after it was taken, the photo editor could provide a simple version for beginners so it's easier to understand, but providing tools for advanced users could provide a lot of freedom, which will make our app more likely for people to share and install. Taking a picture may take a few seconds, but editing a picture could take minutes or an hour, which means more time spent watching ads, and also means more profit.